In [24]:
import sys 
sys.path.append('../utils')
from amg_utils import *
import pandas as pd 
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [2]:
# sql 

fb_tonic_daily_perf_query = """   
   
SELECT    
   a.eventdate,   
   a.buyer_initials,
   a.buyer_name,
   a.account_currency,
   a.account_id,
   a.account_name,
   a.ad_id, 
   a.ad_name,
   a.adset_id,
   a.adset_name,
   a.campaign_id,
   a.campaign_name,
 
   a.job_type,
   d.first_budget,
   COALESCE(a.fb_clicks_all, 0) AS fb_clicks_all,
   COALESCE(a.fb_impressions, 0) AS fb_impressions,
   COALESCE(a.fb_leads, 0) AS fb_leads,
   COALESCE(a.fb_link_click, 0) AS fb_link_click,
   a.fb_spend,
   a.rev_clicks,
   a.gross_revenue,
   a.net_revenue,
   a.fb_clicks_all / COALESCE(NULLIF(a.fb_impressions, 0), 1) AS buy_side_ctr,
   a.net_revenue / COALESCE(NULLIF(a.fb_spend, 0), 1) AS roas,
   a.net_revenue / COALESCE(NULLIF(a.fb_clicks_all, 0), 1) AS rpc,
   (a.fb_spend / COALESCE(NULLIF(a.fb_impressions, 0), 1))*1000 AS buy_side_cpm,
   a.fb_spend / COALESCE(NULLIF(a.fb_clicks_all, 0), 1) AS cpc,
   a.net_revenue / COALESCE(NULLIF(a.rev_clicks, 0), 1) AS rpp,
   a.fb_spend / COALESCE(NULLIF(a.rev_clicks, 0), 1) AS cpp,
   a.net_revenue - a.fb_spend AS contrib,    

   b.budget_type,   
   b.status, 

   c.devices,
   c.countries,
   c.network,
   targeting_json:age_max::string AS age_max,
   targeting_json:age_min::string AS age_min,
   targeting_json:facebook_positions::string AS facebook_positions,
   targeting_json:locales::string AS locales  

FROM PRODUCTION.BD_S2CINTERNAL.v_cm_fb_tonic_daily_combine a 

LEFT JOIN SEM_TOOLS.cm_fb_campaign_management.ad_set_latest b 
   ON a.adset_id = b.id  
   AND a.campaign_id = b.campaign_id 
   AND a.adset_name = b.name

LEFT JOIN PRODUCTION.BD_S2CINTERNAL.V_FACEBOOK_ADSET_METADATA c 
   ON a.adset_id = c.id 
   AND a.eventdate = c.event_date

LEFT JOIN (
   SELECT id, MIN(updated) AS first_updated, MIN(budget) AS first_budget
   FROM sem_tools.CM_FB_CAMPAIGN_MANAGEMENT.ad_set
   GROUP BY id
) AS d ON a.adset_id = d.id

WHERE a.buyer_initials = 'GN'   
and job_type = 'final'

ORDER BY  a.ad_id ASC, a.eventdate ASC  
   """
fb_tonic_daily_perf = eq(fb_tonic_daily_perf_query)

In [ ]:
# fb_tonic_daily_perf.to_csv('fb_tonic_daily_perf.csv', index=False)

In [25]:
# fb_tonic_daily_perf = pd.read_csv("fb_tonic_daily_perf.csv", parse_dates=['eventdate'])

In [26]:
# Crate campaign duration in days variable
# fb_tonic_daily_perf.insert(2, 'campaign_duration', fb_tonic_daily_perf.groupby('campaign_id')['eventdate'].transform(lambda x: (x.max() - x.min()).days))

# # Create campaign end date variable
# fb_tonic_daily_perf.insert(2, 'campaign_end_date', fb_tonic_daily_perf.groupby('campaign_id')['eventdate'].transform("max"))

# fb_tonic_daily_perf.insert(2, 'fb_created', fb_tonic_daily_perf.groupby('campaign_id')['eventdate'].transform("min"))


# # Filter out campaigns with less than {min_campaign_duration} days duration
# min_campaign_duration = 4 
# series = fb_tonic_daily_perf[fb_tonic_daily_perf['campaign_duration'] > 4].copy()
# series.ad_id = series.ad_id.astype('str')

# static_cols = [      
# "network",
# "devices"]   

# def one_hot_encode(df, cols):
#     encoded = pd.get_dummies(df[cols])
#     df = df.drop(columns=cols, axis=1)
#     df = pd.concat([df, encoded], axis=1)
#     return df

# series =  one_hot_encode(series, static_cols)

In [27]:
def update_pause_and_extend_campaign_conditions(series):
    campaign_duration_condition = (series['eventdate'] - series['fb_created']).dt.days >= 7
    roas_condition_pause = series.groupby('ad_id')['roas'].transform(lambda x: (x.shift(1) < 1) & (x.shift(2) < 1) & (x.shift(3) < 1))    
    series['pause_campaign'] = (campaign_duration_condition & roas_condition_pause).fillna(False).astype(bool)    
    series['extend_campaign'] = ~series['pause_campaign']
    
    return series

In [28]:
def update_budgets_based_on_conditions(series):
    series['budget_increase'] = False
    series['budget_decrease'] = False

    series['roas_increase_condition'] = (series.groupby('ad_id')['roas'].shift(1) > 1) & (series.groupby('ad_id')['roas'].shift(2) > 1)
    series['roas_decrease_condition'] = (series.groupby('ad_id')['roas'].shift(1) < 1) & (series.groupby('ad_id')['roas'].shift(2) < 1)

    for ad_id, group in series.groupby('ad_id'):
        prev_budget = None  
        for i, row in group.iterrows():
            if prev_budget is None:
                prev_budget = row['budget']
                continue  

            budget_increase = row['roas_increase_condition'] and prev_budget < 10000
            budget_decrease = row['roas_decrease_condition'] and prev_budget > 1000

            series.at[i, 'budget_increase'] = budget_increase
            series.at[i, 'budget_decrease'] = budget_decrease

            if budget_increase:
                new_budget = prev_budget * 2.0
            elif budget_decrease:
                new_budget = prev_budget / 2.0
            else:
                new_budget = prev_budget

            series.at[i, 'budget'] = new_budget
            prev_budget = new_budget 

    series.drop(['roas_increase_condition', 'roas_decrease_condition'], axis=1, inplace=True)
    return series

In [29]:
def generate_autobid_sequence(series):
    series['budget'] = series.groupby('ad_id')['first_budget'].transform('first').astype(float)
    return (series
            .pipe(update_pause_and_extend_campaign_conditions)
            .pipe(update_budgets_based_on_conditions))

In [30]:
# series = generate_autobid_sequence(series)

In [ ]:
# series.to_csv('series_2.csv', index=False)

In [3]:
series = pd.read_csv("series_2.csv", parse_dates=['eventdate'])

In [5]:
# peek(eq(f"select * from production.external_consoles.facebook_spend_data where adset_id = {series.adset_id[0]} order by ad_id, eventdate, hourly_stats_aggregated_by_advertiser_time_zone"))

In [6]:
fb_hourly_data = eq(f"select * from production.external_consoles.facebook_spend_data where adset_id in ({','.join(map(str, series.adset_id.tolist()))}) order by ad_id, eventdate, hourly_stats_aggregated_by_advertiser_time_zone")

In [8]:
# s = eq("select * from production.external_consoles.facebook_spend_data where adset_id=120205035015970410 order by eventdate, hourly_stats_aggregated_by_advertiser_time_zone")

In [9]:
# peek(s)

In [90]:
# s_pivoted=  pivot_hourly_stats(s)

In [17]:
def pivot_hourly_stats(df):
    """
    Pivot hourly stats for clicks, cpc, cpm, ctr, impressions, spend, and include ad_id into separate columns
    for each hour in 'hourly_stats_aggregated_by_advertiser_time_zone' per 'eventdate' and 'ad_id'.
    
    Parameters:
    - df: DataFrame containing the original data.
    
    Returns:
    - DataFrame with pivoted hourly stats including ad_id, sorted by 'ad_id' and then 'eventdate' at the end.
    """
    # Ensure the 'eventdate' column is of type datetime if not already
    df['eventdate'] = pd.to_datetime(df['eventdate'].dt.date)

    # Group by 'eventdate', 'ad_id', and 'hourly_stats_aggregated_by_advertiser_time_zone' to get counts per hour
    grouped = df.groupby(['eventdate', 'ad_id', 'hourly_stats_aggregated_by_advertiser_time_zone'])

    # Pivot the table for each metric
    metrics = ['clicks', 'cpc', 'cpm', 'ctr', 'impressions', 'spend']
    final_df = pd.DataFrame()  # Initialize an empty DataFrame to hold the result

    for metric in metrics:
        pivoted = grouped[metric].sum().unstack(fill_value=0)
        pivoted.columns = [f"{metric}_{i+1}" for i in range(len(pivoted.columns))]
        if final_df.empty:  # For the first metric, assign the pivoted DataFrame
            final_df = pivoted
        else:  # For subsequent metrics, join with the existing DataFrame
            final_df = final_df.join(pivoted, how='outer')

    # Reset index to bring 'eventdate' and 'ad_id' back as columns
    final_df.reset_index(inplace=True)

    # Sort the final DataFrame by 'ad_id' and then 'eventdate'
    final_df = final_df.sort_values(by=['ad_id','eventdate'])

    return final_df

In [18]:
fb_hourly_data_pivoted = pivot_hourly_stats(fb_hourly_data)

In [21]:
fb_hourly_data_pivoted.iloc[:20]

,eventdate,ad_id,clicks_1,clicks_2,clicks_3,clicks_4,clicks_5,clicks_6,clicks_7,clicks_8,clicks_9,clicks_10,clicks_11,clicks_12,clicks_13,clicks_14,clicks_15,clicks_16,clicks_17,clicks_18,clicks_19,clicks_20,clicks_21,clicks_22,clicks_23,clicks_24,cpc_1,cpc_2,cpc_3,cpc_4,cpc_5,cpc_6,cpc_7,cpc_8,cpc_9,cpc_10,cpc_11,cpc_12,cpc_13,cpc_14,cpc_15,cpc_16,cpc_17,cpc_18,cpc_19,cpc_20,cpc_21,cpc_22,cpc_23,cpc_24,cpm_1,cpm_2,cpm_3,cpm_4,cpm_5,cpm_6,cpm_7,cpm_8,cpm_9,cpm_10,cpm_11,cpm_12,cpm_13,cpm_14,cpm_15,cpm_16,cpm_17,cpm_18,cpm_19,cpm_20,cpm_21,cpm_22,cpm_23,cpm_24,ctr_1,ctr_2,ctr_3,ctr_4,ctr_5,ctr_6,ctr_7,ctr_8,ctr_9,ctr_10,ctr_11,ctr_12,ctr_13,ctr_14,ctr_15,ctr_16,ctr_17,ctr_18,ctr_19,ctr_20,ctr_21,ctr_22,ctr_23,ctr_24,impressions_1,impressions_2,impressions_3,impressions_4,impressions_5,impressions_6,impressions_7,impressions_8,impressions_9,impressions_10,impressions_11,impressions_12,impressions_13,impressions_14,impressions_15,impressions_16,impressions_17,impressions_18,impressions_19,impressions_20,impressions_21,impressions_22,impressions_23,impressions_24,spend_1,spend_2,spend_3,spend_4,spend_5,spend_6,spend_7,spend_8,spend_9,spend_10,spend_11,spend_12,spend_13,spend_14,spend_15,spend_16,spend_17,spend_18,spend_19,spend_20,spend_21,spend_22,spend_23,spend_24
0,2024-01-17,120203732424570410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.000,0.000,0.000000,0.00,0.000000,0.00,0.000000,0.146,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.172414,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,17.241379,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.73,0.00,0.00,0.00,0.00,0.00
9,2024-01-18,120203732424570410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.3400,0.000,0.000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,23.684211,38.750000,6.666667,30.000000,0.000000,13.333333,26.153846,0.000000,26.000000,28.000000,15.000000,32.500000,34.000000,0.000000,0.000000,8.333333,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.692308,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,19.0,8.0,3.0,3.0,0.0,6.0,13.0,0.0,5.0,5.0,6.0,4.0,5.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.45,0.31,0.02,0.09,0.00,0.08,0.34,0.00,0.13,0.14,0.09,0.13,0.17,0.00,0.00,0.05,0.00,0.00,0.00,0.00
31,2024-01-19,120203732424570410,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.2200,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.000,0.000,0.000000,0.00,0.116667,0.13,0.000000,0.000,0.00,0.00,0.00,0.0,0.0,0.0,20.0,40.0,31.428571,48.000000,56.666667,30.000000,0.000000,0.000000,90.000000,0.000000,0.000000,0.000000,20.000000,30.000000,23.333333,37.142857,0.000000,20.000000,10.000000,30.00,0.000000,0.0,35.0,0.0,0.0,0.0,14.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,20.000000,28.571429,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,2.0,7.0,5.0,3.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,15.0,7.0,0.0,2.0,2.0,1.0,0.0,0.0,2.0,0.00,0.02,0.08,0.22,0.24,0.17,0.12,0.00,0.00,0.27,0.00,0.00,0.00,0.02,0.03,0.35,0.26,0.00,0.04,0.02,0.03,0.00,0.00,0.07
53,2024-01-20,120203732424570410,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,

In [22]:
series_with_fb_hourly_data = pd.merge(series, fb_hourly_data_pivoted, on=['ad_id', 'eventdate'], how='left')

In [23]:
peek(series_with_fb_hourly_data.iloc[:20])

,,,,,,,,,,,,,,,,,,,,,,
0,eventdate,datetime64[ns],2024-01-18 00:00:00,2024-01-19 00:00:00,2024-01-20 00:00:00,2024-01-21 00:00:00,2024-01-22 00:00:00,2024-01-23 00:00:00,2024-01-24 00:00:00,2024-01-25 00:00:00,2024-01-18 00:00:00,2024-01-19 00:00:00,2024-01-20 00:00:00,2024-01-21 00:00:00,2024-01-22 00:00:00,2024-01-23 00:00:00,2024-01-24 00:00:00,2024-01-25 00:00:00,2024-01-26 00:00:00,2024-01-18 00:00:00,2024-01-20 00:00:00,2024-01-21 00:00:00
1,buyer_initials,object,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN,GN
2,fb_created,object,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18,2024-01-18
3,campaign_end_date,object,2024-01-25,2024-01-25,2024-01-25,2024-01-25,2024-01-25,2024-01-25,2024-01-25,2024-01-25,2024-01-26,2024-01-26,2024-01-26,2024-01-26,2024-01-26,2024-01-26,2024-01-26,2024-01-26,2024-01-26,2024-01-24,2024-01-24,2024-01-24
4,campaign_duration,int64,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,6,6,6
5,buyer_name,object,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets,Golden Nuggets
6,account_currency,object,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
7,account_id,int64,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781,7150655894950781
8,account_name,object,fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles),fw84 - FB SCMT 1016 GN (America/Los_Angeles)
9,ad_id,int64,120203732424570410,120203732424570410,120203732424570410,120203732424570410,120203732424570410,120203732424570410,120203732424570410,120203732424570410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732424650410,120203732425100410,120203732425100410,120203732425100410


In [25]:
def generate_word2vec_model(keywords, vector_size=100, window=5, min_count=1, workers=4):
    """
    Generate a Word2Vec model given a list of keywords.
    
    :param keywords: List of keywords (strings).
    :param vector_size: Dimensionality of the word vectors.
    :param window: Maximum distance between the current and predicted word within a sentence.
    :param min_count: Ignores all words with total frequency lower than this.
    :param workers: Use these many worker threads to train the model.
    :return: Trained Word2Vec model.
    """
    # Tokenize each keyword into words
    tokenized_keywords = [word_tokenize(keyword.lower()) for keyword in keywords]
    
    # Create and train the Word2Vec model
    model = Word2Vec(sentences=tokenized_keywords, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    
    return model

In [26]:
get_tonic_kws_query = """
   
select 
   
keyword,
count(1),
sum(clicks), 
sum(actual_net_revenue),
sum(rpc)
from 
PRODUCTION.BD_S2CINTERNAL.V_CM_TONIC_KEYWORD_DATA a 
left join 

(select 
        *,
        row_number() over (partition by id order by updated desc nulls last) as rn
    from SEM_TOOLS.CM_FB_CAMPAIGN_MANAGEMENT.TONIC_CAMPAIGN
    qualify rn = 1) b 

on a.tonic_campaign_name = b.name

where b.country_code = 'US' 

group by keyword order by 2 desc , 3 desc
"""

all_tonic_kws = eq(get_tonic_kws_query)

In [29]:
list_tonic_kws= all_tonic_kws.keyword.tolist()

In [30]:
model = generate_word2vec_model(list_tonic_kws)

In [31]:
import pickle

with open('word2vec_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [40]:
new_keyword = "Budget Tires"
tokenized_new_keyword = word_tokenize(new_keyword.lower())
for word in tokenized_new_keyword:
    if word in model.wv:
        print(model.wv[word])

[ 3.8072038e-02 -2.1931203e-02  2.2519477e-02 -1.4492852e-02
 -2.4481924e-02 -2.9446300e-02 -1.6092477e-04  2.1039206e-01
 -1.5000073e-02 -1.0968311e-02 -4.7951131e-03 -7.4388571e-02
 -7.2982728e-02  3.4980014e-02 -2.5672927e-02 -2.8654220e-02
 -6.2845364e-02 -5.7231229e-02  4.7858521e-02 -2.4300609e-02
 -2.6539357e-02 -2.7446104e-02  3.6074709e-02 -5.3596981e-02
  5.4472264e-02  5.1000457e-02  4.3151349e-02 -2.3279084e-02
 -4.7569975e-02  5.7110428e-03  2.2428704e-02 -5.8855079e-02
 -1.8831382e-02 -2.2133177e-02 -3.6000725e-02  5.9586868e-02
  5.1091302e-02 -2.2930821e-02 -1.7603515e-02 -4.7382001e-02
 -5.4035923e-03 -7.4360888e-03 -4.2852055e-02  5.8000267e-02
  8.3569542e-02  5.5650506e-02  2.5406444e-02  6.9446817e-02
  6.9341354e-02 -4.1457660e-02  1.7691506e-02 -5.5096000e-02
 -1.4189494e-02 -5.0320689e-02 -2.6547451e-02 -6.0190088e-03
  1.2212300e-02 -7.0767857e-02 -9.5286980e-02  4.4282727e-02
 -2.6046004e-02  5.2431487e-02  3.9349589e-02 -5.2859817e-02
 -2.6176907e-02 -1.38588

In [41]:
def get_keyword_vector(model, keyword):
    tokenized_keyword = word_tokenize(keyword.lower())
    keyword_vector = np.zeros(model.vector_size)
    valid_tokens = 0
    for word in tokenized_keyword:
        if word in model.wv:
            keyword_vector += model.wv[word]
            valid_tokens += 1
    if valid_tokens > 0:
        keyword_vector /= valid_tokens
    return keyword_vector

series_with_fb_hourly_data['kw_vector'] = series_with_fb_hourly_data['keyword'].apply(lambda x: get_keyword_vector(model, x))

KeyError: 'keyword'

In [ ]:
series_with_fb_hourly_data